In [ ]:
import geemap
import ee
from ee import batch

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import calendar
import altair as alt
from altair_saver import save

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
center_lat = 22.62739 
center_lon = 77.17170
zoomlevel=6

viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2020-01-01","2020-12-31").select('avg_rad').median()
map1 = geemap.Map(center=[center_lat, center_lon],zoom=4)
map1.addLayer(viirs, {}, "VIIRS-DNB- India 2020", opacity=0.9)
map1.addLayerControl()
map1

In [ ]:
hkv_lat = 28.554769 - 8
hkv_lon = 77.194431 + 1
#delhi_aoi = ee.Geometry.Point([hkv_lon, hkv_lat]).buffer(1500000);
india_aoi = ee.Geometry.Rectangle([67, 4 , 97, 35])
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2017-01-01","2017-12-31").select('avg_rad').median()
clipped = viirs.clip(india_aoi)
map2 = geemap.Map(center=[hkv_lat, hkv_lon],zoom=4)
map2.add_basemap('SATELLITE')
map2.addLayer(clipped, {}, "VIIRS-DNB- HKV 2020")
map2.addLayerControl()
map2

In [ ]:
# Viz 2: India GIF
years = [str(i) for i in range(2015,2021)]

for year in years:
    print(year)
    start = year + "-01-01"
    end = year + "-12-31"
    viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(start, end).select('avg_rad').median()
    clipped = viirs.clip(india_aoi)
    
    task = batch.Export.image.toDrive(image=clipped,
                                     fileFormat='GEO_TIFF',
                                     description=year,
                                     folder='tmp2',
                                     scale = 800,
                                     maxPixels=1e13)

    task.start()


In [ ]:
# Viz 3: SRK's mansion
poi = ee.Geometry.Point(72.820378, 19.045113).buffer(500) #72.820378
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate('2015-01-01','2019-12-31')

def poi_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=30).get('avg_rad')
    return img.set('date', img.date().format()).set('mean',mean)

poi_reduced_imgs = viirs.map(poi_mean)

nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)
df = pd.DataFrame(nested_list.getInfo(), columns=['date','mean'])

df['date'] = pd.to_datetime(df['date'])
df['month-year']= df.date.dt.month.apply(lambda x: calendar.month_abbr[x]) +'-'+ df.date.dt.year.astype(str)
df['order'] = range(len(df))
#df = df.set_index('month-year')

#fig, ax = plt.subplots(figsize=(15,7))
#plt.xticks(rotation = 65)
# we'll create the plot by setting our dataframe to the data argument
#sns.lineplot(data=df, ax=ax)

# we'll set the labels and title
#ax.set_ylabel('Mean radiance',fontsize=20)
#ax.set_xlabel('Month-year',fontsize=20)
#ax.set_title('Monthly mean radiance for Bandra West (Jan 2015 to Dec 2019)',fontsize=20);

In [ ]:
line_chart = alt.Chart(df, title = 'NL in Bandra West, Mumbai').mark_line().encode(
    x=alt.X('month-year:T', axis=alt.Axis(title='Month-Year')),
    y=alt.Y('mean:Q', axis=alt.Axis(title='Mean Night Light Intensity'))
)
save(line_chart, 'Viz4.html')

In [ ]:
# Viz 4: Daytime imagery Delhi Airport
collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')

## Above commented out as you might not know the path and row this allows you to pass a coordinate
pointUAE = ee.Geometry.Point(77.194431, 28.554769)
pathrow = collection.filterBounds(pointUAE)
 
##Filter cloudy scenes.
clouds = pathrow.filter(ee.Filter.lt('CLOUD_COVER', 5))

## select the bands, we are going for true colour... but could be any!
bands = clouds.select(['B4', 'B3', 'B2'])

##make the data 8-bit.
def convertBit(image):
    return image.multiply(512).uint8()  

## call the conversion    
outputVideo = bands.map(convertBit)

#Export to video.
# Delhi
out = batch.Export.video.toDrive(outputVideo, description='delhi_video', dimensions = 720, framesPerSecond = 2, region=([77.21,28.6], [77.06, 28.6], [77.06, 28.52],[77.21, 28.52]), maxFrames=10000)

## process the image
process = batch.Task.start(out)


In [ ]:
# Viz 5: HKV lights during pandemic
poi = ee.Geometry.Point([hkv_lon, hkv_lat]).buffer(6000);
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate('2018-06-01','2020-12-31')

def poi_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=30).get('avg_rad')
    return img.set('date', img.date().format()).set('mean',mean)

poi_reduced_imgs = viirs.map(poi_mean)

nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)
df = pd.DataFrame(nested_list.getInfo(), columns=['date','mean'])

df['date'] = pd.to_datetime(df['date'])
df['month-year']= df.date.dt.month.apply(lambda x: calendar.month_abbr[x]) +'-'+ df.date.dt.year.astype(str)
df = df.set_index('month-year')

fig, ax = plt.subplots(figsize=(15,7))
plt.xticks(rotation = 65)
# we'll create the plot by setting our dataframe to the data argument
sns.lineplot(data=df, ax=ax)

# we'll set the labels and title
ax.set_ylabel('Mean radiance',fontsize=20)
ax.set_xlabel('Month-year',fontsize=20)
ax.set_title('Monthly mean radiance for HKV (Jun 2019 to Dec 2020)',fontsize=20);